# SLICEop
Operational forecast of the St. Lawrence river's freeze-up date near Montreal, QC, Canada.  
This is the operational version of [SLICE](https://github.com/McGill-sea-ice/SLICE) the forecast developped and tested by Amélie Bouchat.

---

# Documentation and Usage

## Method

We use time series of September total cloud cover, November snowfall and December 2 m – air temperature from the [ECMWF's seasonal forecast system](https://www.ecmwf.int/en/forecasts/documentation-and-support/long-range)
to perform a multiple linear regression that predicts the date on which the water temperature near Montreal drops to the freezing point and ice formation begins. 

<div>
    <img src="Fig01.png" width=800/>
</div>  

*Fig. 1: Schematic of the regression model.*

The predictor variables are extracted from the [SEAS5.1 seasonal forecast](https://www.ecmwf.int/en/elibrary/80921-seas5-and-future-evolution-long-range-forecast-system) starting in July of each year. Once data for a given predictor variables becomes available from ERA5, the SEAS5.1 forecast data is replaced by the equivalent [ERA5 reanalysis](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5) data: *e.g.* in mid October, the total cloud cover for September is available from ERA5 and is used to do the forecast instead of the forecasted total cloud cover.  
The water temperature and actual date of freeze-up are observed at the [Longueuil water treatment plant](https://www.longueuil.quebec/fr/eau-potable) near Montreal. The river is assumed to start freezing up when the water temperature at the water treatment plant falls below 0.75$^{\circ}$C, which has been shown to correlate best with observed freeze-up dates from other sources like the [St-Lawrence Seaway Management Corporation (SLSMC)](https://greatlakes-seaway.com/en/) or ice charts based on reconnaissance flights by the [Canadian Coast Guard](https://www.ccg-gcc.gc.ca/index-eng.html). 
The three predictor variables have been found to best predict the freeze-up date based on [extensive testing by Amélie Bouchat](https://github.com/McGill-sea-ice/SLICE).
They also tested the use of machine learning for the forecast, which was found to perform worse on medium to seasonal time scales.

### Automated tasks

#### Daily

1. [**`daily_Twater.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/daily_Twater.py) Water temperature is tranferred from the water treatment plant daily with a temporal resolution of one minute. The daily average is computed and added to the time series (`Twater_Longueuil_updated.nc`) after a basic quality control to remove outliers that are either warmer than 30$^{\circ}$C or exhibit a spike in temperature (defined as a temperature change of more than 1$^{\circ}$C per hour). 
2. [**`daily_MODIS.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/daily_MODIS.py)The most recent Terra/MODIS True Color satellite image for the greater Montreal region is downloaded from [NASA worldview](https://worldview.earthdata.nasa.gov).

<div>
    <img src="https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2025-01-20T00:00:00Z&BBOX=45.1855,-74.2417,45.8297,-73.261&CRS=EPSG:4326&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,Coastlines_15m&WRAP=day,x&FORMAT=image/jpeg&WIDTH=893&HEIGHT=586&colormaps=,&ts=1744122633289" width="500"/>
</div>

*Fig. 2: Example of the Terra/MODIS image.*

#### Monthly

1. [**`monthly_SEAS51_ERA5.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/monthly_SEAS51_ERA5.py) The ECMWF's SEAS5.1 seasonal forecast is run every 1st of the month and the data made available on the 5th. Starting in July, we try to download the desired variables on the 7th (to allow for a small buffer of 2 days in case of delays). Once past the month of a specific variable (September for total cloud cover etc.), we download the ECMWF's ERA5 reanalysis data instead to benefit from the actual observations incorporated into ERA5.
2. [**`monthly_preprocess.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/monthly_preprocess.py) The preprocessing consists mainly of the computation of monthly averages (September total cloud cover, December 2 m – air temperature) and sums (November snowfall). For SEAS5.1 the preprocessing is done for each ensemble member and the ensemble mean.
3. [**`monthly_forecast.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/monthly_forecast.py) The regression model built from the yearly time series of the predictor variables is used with the monthly averages/sums from SEAS5.1 and/or ERA5 in order to predict the freeze-up date. If SEAS5.1 was used to compute the monthly averages/sums, the forecast is performed for each ensemble member and the ensemble mean.

#### Weekly

<div>
    <img src="Fig03.png" width=800/>
</div>  

*Fig. 3: Schematic timeline of the dates the forecast is performed. Each orange x indicates the monthly forecast, each vertical black line indicates the weekly forecast. In the black boxes is indicated for each monthly forecast whether each of the input variables originate from SEAS5.1 or ERA5.*

1. [**`weekly_ERA5.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/weekly_ERA5.py) Each Monday during the months of September, November, and December and up to the 6th of the following month, we check whether ERA5 data is available to replace the SEAS5.1 data and if yes, download the respective variable.
2. [**`weekly_preprocess.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/weekly_preprocess.py) If ERA5 data was downloaded in the step before, here we compute a monthly average or sum based on the combination of ERA5 and SEAS5.1 data. The preprocessing is performed for each ensemble member and the ensemble mean of SEAS5.1. If applicable, the SEAS5.1 data in each of the ensemble members is replaced by the same ERA5 data. 
3. [**`weekly_forecast.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/weekly_forecast.py) The forecast is performed exactly the same way as before for the monthly forecast.

<div>
    <img src="Fig04.png" width=1100/>
</div>  

*Fig. 4: Examples of how SEAS5.1 data is replaced by ERA5 data before computing the monthly average or sum during the weekly preprocessing.*

#### Yearly

1. [**`yearly_preprocess.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/yearly_preprocess.py) Once a year in June, the time series shown in Fig. 1 are updated with the previous year's values: The monthly averages for September total cloud cover and December 2 m -- air temperature, the monthly sum for November snowfall and the freeze-up date calculated from the water temperature at Longueuil.

---

## Usage

### Preparation

After cloning the repository

```
git clone git@github.com:McGill-sea-ice/SLICEop.git
```

the two main requirements are

1. having an account with [Copernicus](https://www.copernicus.eu/en) to access the [ECMWF](https://www.ecmwf.int/)'s data through their [Climate Data Store](https://cds.climate.copernicus.eu/), to be used by the python module `cdsapi`. Follow the instructions under `1. Setup the CDS API personal access token` on [https://cds.climate.copernicus.eu/how-to-api](https://cds.climate.copernicus.eu/how-to-api) and then
2. create a python environment with the required packages specified in [`SLICEop/environment.yml`](https://github.com/McGill-sea-ice/SLICEop/blob/main/environment.yml), e.g. with
   
    ```
    conda env create -f environment.yml
    ```

    If you choose another way to create the environment with conda, make sure that the environment is called `sliceop`. If you create the environment in another way, like `virtualenv`,     make sure the environment is loaded by changing the lines under `#load conda environment` in `SLICEop/SLICEop/auto/run_*.sh`.

Further make sure to update the paths in [`SLICEop/setup.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/setup.sh) and run `source setup.sh` to set the environment variables.

### Initialization

The initialization includes a large number of downloads from the Climate Data Store and can take a long time (on the order of days).  

Run 

```
source setup.sh
```

if not already done and then run

```
SLICEop/SLICEop/init.sh
```

[`init.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/init.sh) will create a couple of files that are needed later on, then call [`SLICEop/SLICEop/downloads/initial_download_ERA5.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/initial_download_ERA5.py) to download the ERA5 data required from previous years. Depending on how busy the Climate Data Store's server is, this could take a long time. The time period and region downloaded are all specified in `initial_download_ERA5.py`.
Lastly, `init.sh` will call [`SLICEop/SLICEop/downloads/initial_Twater.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/initial_Twater.py) do initialize the time series of water temperature from the Longueuil water treatment plant. Additionally, the time series of freeze-up dates is computed here. This script is highly specialized to work with this specific time series as there are gaps to fill, different data formats to account for etc. If you want to use a different time series of water temperature, this script will be of little use.



### Testing

Once the initialization done, there is the possibility to run a test that simulates the weekly and monthly forecasts for the period of past years specified in [`SLICEop/SLICEop/run_test.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/run_test.sh).  

Running this script will generate two files per simulated forecast season starting in year `YYYY`, one (`YYYYFUDmonthly`) with all the monthly forecasts and one (`YYYYFUDweekly`) with all the weekly forecasts.  
The generated files will have the format:
```
time,number,FUD
1997-07-07,0,359
1997-07-07,1,353
1997-07-07,2,346
...
```
The first column (`time`) represents the date the forecast was run. The second column `number` is the number of the ensemble member, where `0` represents either the ensemble mean or is the only number present when there is no ensemble (ERA5 data as input to the forecast). The third column (`FUD`) is the predicted freeze-up day of the year, where all years are assumed to have 365 years, e.g. December 22 is always `FUD = 356`. `FUD > 365` represent a freeze-up date in the next year on day `FUD - 365`.  
No figures will be generated during the test.

### Operational use